## Train mode

Get dataset from [https://www.kaggle.com/moltean/fruits](https://www.kaggle.com/moltean/fruits)

In [ ]:
import tensorflow as tf

In [ ]:
tf.config.experimental.list_physical_devices('GPU')

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import RMSprop, SGD, Adam
from keras import backend as K

import keras

import matplotlib.pyplot as plt

In [ ]:
import os
data_dir = './'

In [ ]:
train_data_dir = os.path.join(data_dir, 'train')
validation_data_dir = os.path.join(data_dir, 'val')
print(train_data_dir, os.listdir(train_data_dir))
print(validation_data_dir, os.listdir(validation_data_dir))
len(os.listdir(train_data_dir))

In [ ]:
from PIL import Image

img = Image.open(train_data_dir + '/Banana/r_99_100.jpg')
plt.imshow(img)
print(img.size)

In [ ]:
img_width, img_height = 224, 224
epochs = 5 #25
batch_size = 16 #64
numclasses = len(os.listdir(train_data_dir))

In [ ]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    zoom_range = 0.2, # Randomly zoom image 
    shear_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
nb_train_samples = train_generator.n #3362
nb_validation_samples = validation_generator.n #1127

In [ ]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [ ]:
def resnet50tl(input_shape, outclass, sigma):
       
    base_model = keras.applications.resnet50.ResNet50(weights='imagenet', include_top=False)
           
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    
    prediction = Dense(outclass, activation=sigma)(x)
    model = Model(inputs=base_model.input, outputs=prediction)
    
    return model

In [ ]:
model = resnet50tl(input_shape, numclasses, 'softmax')
lr = 1e-5
decay = 1e-7 #0.0
#optimizer = RMSprop(lr=lr, decay=decay)
optimizer = SGD(lr=1e-4, momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint

filepath="fruits-{epoch:02d}-{loss:.4f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min', period=5)
callbacks_list = [checkpoint]

In [ ]:
import tensorflow as tf
with tf.device("/device:GPU:0"):
    history = model.fit_generator(
        train_generator,
        epochs=epochs,
        shuffle = True, verbose = 1,
        validation_data = validation_generator,
        callbacks=callbacks_list)

In [ ]:
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from io import BytesIO
import os

labels = ['Apple Red 1', 'Avocado', 'Banana', 'Cherry 1', 'Mango', 'Orange', 'Peach']
#test_imgs = ['/Banana/r_45_100.jpg', '/Banana/105_100.jpg', '/Cherry 1/325_100.jpg', '/Peach/321_100.jpg']
test_imgs = ['/Banana/0_100.jpg', '/Banana/122_100.jpg', '/Cherry/103_100.jpg', '/Peach/105_100.jpg']

for test in test_imgs:
    test_img = train_data_dir + test
    img = image.load_img(test_img, target_size=(img_width, img_height))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x /= 255.
    classes = model.predict(x)
    result = np.squeeze(classes)
    result_indices = np.argmax(result)
    
    #img = Image.open(test_img)
    #plt.axis('off')
    #plt.title("{}, {}, {:.2f}%".format(test, labels[result_indices], result[result_indices]*100))
    #plt.imshow(img)
    
    print("{}, {}, {:.2f}%".format(test, labels[result_indices], result[result_indices]*100))

In [ ]:
# Get training and test loss histories
training_loss = history.history['loss']
training_acc = history.history['accuracy']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

fig=plt.figure(figsize=(12, 4))
# Visualize loss history
fig.add_subplot(121)
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, training_acc, 'b-')
plt.legend(['Training Loss', 'Training Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Training Loss/Acc')

# Get training and test loss histories
val_acc = history.history['val_accuracy']
training_acc = history.history['accuracy']

# Create count of the number of epochs
epoch_count = range(1, len(val_acc) + 1)

# Visualize loss history
fig.add_subplot(122)
plt.plot(epoch_count, val_acc, 'r--')
plt.plot(epoch_count, training_acc, 'b-')
plt.legend(['Validation Accuracy', 'Training Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

plt.show();